<a href="https://colab.research.google.com/github/zeliasporto/game_rec/blob/main/steamdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests pandas tqdm

In [ ]:
import requests
import pandas as pd
from tqdm import tqdm
import time

# 1. Obter todos os appids
print("Baixando lista de appids...")
applist_url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
response = requests.get(applist_url)
apps = response.json()['applist']['apps']

# 2. Prepara a coleta dos dados
dados = []

print("Coletando dados dos jogos...")

# 3. Iterar sobre os appids com barra de progresso
for app in tqdm(apps):
    appid = app['appid']
    name = app['name']

    if not name.strip():
        continue  # pular apps sem nome

    details_url = f"https://store.steampowered.com/api/appdetails?appids={appid}"
    try:
        r = requests.get(details_url, timeout=5)
        r.raise_for_status()
        data = r.json()

        if not data.get(str(appid), {}).get('success'):
            continue

        game_data = data[str(appid)]['data']
        if game_data.get('type') != 'game':
            continue

        genres = [g['description'] for g in game_data.get('genres', [])]
        categories = [c['description'] for c in game_data.get('categories', [])]

        dados.append({
            'appid': appid,
            'name': name,
            'genres': ', '.join(genres),
            'categories': ', '.join(categories)
        })

    except Exception as e:
        continue  # pular erros de timeout ou estrutura

    time.sleep(0.3)  # respeitar limite de requisições

# 4. Salvar como CSV
print(f"\nTotal de jogos coletados: {len(dados)}")
df = pd.DataFrame(dados)
df.to_csv("jogos_steam.csv", index=False, encoding='utf-8-sig')
print("Arquivo 'jogos_steam.csv' salvo com sucesso.")


Baixando lista de appids...
Coletando dados dos jogos (isso pode levar um tempo)...


100%|██████████| 253622/253622 [1:59:11<00:00, 35.47it/s]


Total de jogos coletados: 1197
Arquivo 'jogos_steam.csv' salvo com sucesso.
